Import packages

In [9]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import sys
from astropy.table import Table, Column
from scipy import stats
from multiprocessing import Pool
from scipy.stats import norm
import os
import glob
from scipy.ndimage import maximum_filter, label, center_of_mass
from matplotlib.backends.backend_pdf import PdfPages
import re
from matplotlib import colors
from concurrent.futures import ProcessPoolExecutor, as_completed
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing
import jupyter
from numpy.fft import rfft2, irfft2
from numpy.fft import fft2, ifft2
from scipy.signal import fftconvolve
import pickle
import umap
import matplotlib.patches as mpatches
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools

Detect all csv files

In [10]:
# detect all csv files in the folder and save their filenames in a list

def get_csv_filenames(folder_path):
  csv_filenames = glob.glob(os.path.join(folder_path, '*.csv'))
  file_names = [os.path.basename(file).replace('.csv', '') for file in csv_filenames]
  return file_names


folder_path = 'c:\\Users\\skyma\\Desktop\\t1\\csv'
file_names = get_csv_filenames(folder_path)
main_path = 'c:\\Users\\skyma\\Desktop\\t1'

print("File names are:", file_names)

File names are: ['sep10000px', 'sep12000px', 'sep2000px', 'sep4000px', 'sep6000px', 'sep8000px']


Set constants

In [11]:
#=======================
# Constants
# bin_size_list = [1000]
bin_size_list = [500, 1000]

#=======================
# Script

filename = None
# Rs_input_list = [float(10000)]
Rs_input_list = [float(5000), float(10000), float(20000)]
cpu_num = int(12)
E_maps = []

Define functions

In [12]:
# =======================
# Filter Definitions
# =======================

def normalize_filter(weight_array, epsilon=1e-10):
    """
    Normalize a filter weight array so that its sum equals 1, with a small threshold to prevent division by zero.

    Parameters:
    - weight_array (numpy array): original filter weights
    - epsilon (float): small value to prevent division by zero if sum is too small

    Returns:
    - normalized_weight_array (numpy array): filter weights normalized to sum = 1
    """
    total = np.sum(weight_array)
    if total < epsilon:
        raise ValueError(f"Cannot normalize: filter weights sum to {total}, which is below the threshold {epsilon}.")
    return weight_array / total

def smooth_center_taper(x, epsilon=0.00001):
    return x**2 / (x**2 + epsilon)

def schirmer_weight(x):

    a, b, c, d, xc = 6., 150., 47., 50., 0.15
    Q = 1. / (1. + np.exp(a - b * x) + np.exp(d * x - c))

    with np.errstate(divide='ignore', invalid='ignore'):
        fx = np.tanh(x / xc) / (x / xc)
        fx[np.isnan(fx)] = 1.0

    return Q * fx

def gaussian_weight(x):
    Q = np.exp(-x**2 / (2 * 0.3 ** 2)) # (std = 0.3)
    return Q

def shifted_gaussian_weight(x, mu=0.5, sigma=0.3):
    """
    Shifted Gaussian filter centered at x=mu (default 0.5).
    """
    Q = np.exp(-((x - mu)**2) / (2 * sigma**2))
    return Q

def top_hat_weight(x, smoothing_width=0.05):
    """
    A smoothed version of the top-hat filter.
    x = r/Rs
    smoothing_width: width over which to smooth the transition (default 10% of Rs)
    """
    Q = np.ones_like(x)
    mask = (x > 1.0) & (x <= 1.0 + smoothing_width)
    Q[mask] = 1.0 - (x[mask] - 1.0) / smoothing_width
    Q[x > 1.0 + smoothing_width] = 0.0
    return Q

def nfw_matched_weight(x):
    Q = np.zeros_like(x)

    eps = 1e-6

    x = np.clip(x, eps, None)

    mask1 = (x < 1)
    mask2 = (x > 1)
    mask3 = (x == 1)

    # r < 1
    Q[mask1] = (2 / (x[mask1]**2 - 1)) * (1 - (2 / np.sqrt(1 - x[mask1]**2)) * np.arctanh(np.sqrt((1 - x[mask1])/(1 + x[mask1]))))

    # r > 1
    Q[mask2] = (2 / (x[mask2]**2 - 1)) * (1 - (2 / np.sqrt(x[mask2]**2 - 1)) * np.arctan(np.sqrt((x[mask2] - 1)/(x[mask2] + 1))))

    # r == 1
    Q[mask3] = 10/3 - 4*np.log(2)

    return Q

def no_filter_weight(x):
    return np.ones_like(x)

filter_bank = {
    "schirmer": schirmer_weight,
    "gaussian": gaussian_weight,
    "shifted_gaussian": shifted_gaussian_weight,
    "tophat": top_hat_weight,
    "nfw": nfw_matched_weight,
    "none": no_filter_weight,
}


# =======================
# Inversion techniques
# =======================

def kaiser_squires_inversion(e1_map, e2_map):
    """
    restores surface density convergence map (kappa) from e1, e2 shear maps using Kaiser-Squires inversion.
    """
    e1_map = np.nan_to_num(e1_map, nan=0.0)
    e2_map = np.nan_to_num(e2_map, nan=0.0)
    ny, nx = e1_map.shape
    kx = np.fft.fftfreq(nx).reshape(1, nx) * 2 * np.pi
    ky = np.fft.fftfreq(ny).reshape(ny, 1) * 2 * np.pi

    k_squared = kx**2 + ky**2
    k_squared[0, 0] = 1.0

    e1_ft = np.fft.fft2(e1_map)
    e2_ft = np.fft.fft2(e2_map)

    D = (kx**2 - ky**2 + 2j * kx * ky) / k_squared
    kappa_ft = (e1_ft + 1j * e2_ft) * D.conjugate()

    kappa = np.fft.ifft2(kappa_ft).real

    return kappa


def plot_kappa_map(kappa_map, tag):
    plt.figure(figsize=(6,5))
    vmax = np.nanmax(np.abs(kappa_map))
    plt.imshow(np.flipud(kappa_map), cmap="bwr", vmin=-vmax, vmax=vmax)
    plt.colorbar(label=r"$\kappa$")
    plt.title(f"{tag} | Surface Density Map ($\\kappa$)")
    plt.tight_layout()
    plt.savefig(f"{folder_name}/kappa_map/{tag}_Rs{int(Rs_input)}_bin{int(bin_size)}_kappa_map.png")
    plt.close()


# =======================
# Define get_bin_stat
# =======================

def get_bin_stat(arr, stat="mean"):

    x_bin = np.arange(x_min, x_max+bin_size, bin_size)
    y_bin = np.arange(y_min, y_max+bin_size, bin_size)

    statistic, x_edge, y_edge, binnumber = stats.binned_statistic_2d(
                                    x, y,
                                    arr,
                                    statistic=stat,
                                    bins=[x_bin, y_bin],
                                )

    return statistic.T


def bin_sigmae(sigmae_arr, stat="mean"):
    """
    按2D分箱把sigmae加权/平均到像素格点。
    """
    x_bin = np.arange(x_min, x_max + bin_size, bin_size)
    y_bin = np.arange(y_min, y_max + bin_size, bin_size)
    statistic, _, _, _ = stats.binned_statistic_2d(
        x, y, sigmae_arr, statistic=stat, bins=[x_bin, y_bin]
    )
    return statistic.T


# ==========================
# General M_ap Computing
# ==========================

def compute_M_ap_at_pixel(ind, filter_name, Rs):
    """
    ind: (row, col)
    filter_name: 字符串；在函数内部用 filter_bank[filter_name] 拿到真正的核函数
    Rs: 像素半径（或已换算好的 Rs_pix）
    """
    row, col = ind
    r = np.sqrt((xv - col)**2 + (yv - row)**2)
    x = r / Rs

    # 在这里把名字变成函数，避免把函数对象跨进程/线程传参
    f = filter_bank[filter_name]

    # 'none' 代表常数权重（不做 smooth taper）；其他核做中心平滑
    if filter_name == "none":
        weight = f(x)                  # no_filter_weight(x) -> 全 1
    else:
        weight = f(x) * smooth_center_taper(x)

    weight = normalize_filter(weight)

    dx, dy = xv - col, yv - row
    angle = np.arctan2(dy, dx)

    et = - e1_binned * np.cos(2 * angle) - e2_binned * np.sin(2 * angle)
    ex = + e1_binned * np.sin(2 * angle) - e2_binned * np.cos(2 * angle)

    M_ap_E_tmp = np.nanmean(weight * et)
    M_ap_B_tmp = np.nanmean(weight * ex)
    return M_ap_E_tmp, M_ap_B_tmp


def calculate_mass_map(filter_name="schirmer"):
    assert filter_name in filter_bank, f"Filter '{filter_name}' not defined. Available: {list(filter_bank.keys())}"
    print(f"Using filter: {filter_name}")

    coord_list = list(zip(yv.flatten(), xv.flatten()))
    # 先用顺序列表推导（稳定）；需要再提速我们再换更快的实现
    res = [compute_M_ap_at_pixel(ind, filter_name, Rs) for ind in coord_list]

    M_ap_E = np.array(res)[:, 0].reshape((nrow, ncol))
    M_ap_B = np.array(res)[:, 1].reshape((nrow, ncol))
    return M_ap_E, M_ap_B


# ============================
# Define plot_E_B function
# ============================

def plot_E_B(E_mat, B_mat, tag, zero=False):

    threshold = np.max(
                        [
                            np.abs( np.nanmin(E_mat[np.isfinite(E_mat)]) ),
                            np.abs( np.nanmax(E_mat[np.isfinite(E_mat)]) ),
                            np.abs( np.nanmin(B_mat[np.isfinite(B_mat)]) ),
                            np.abs( np.nanmax(B_mat[np.isfinite(B_mat)]) ),
                        ]
                    )

    negative_threshold = np.min(
                                  [
                                      np.abs( np.nanmin(E_mat[np.isfinite(E_mat)]) ),
                                      np.abs( np.nanmax(E_mat[np.isfinite(E_mat)]) ),
                                      np.abs( np.nanmin(B_mat[np.isfinite(B_mat)]) ),
                                      np.abs( np.nanmax(B_mat[np.isfinite(B_mat)]) ),
                                  ]
                    )



    if zero:
        threshold_upper = threshold
        threshold_lower = 0.
    else:
        threshold_upper = threshold
        threshold_lower = -threshold



    print("Plotting E B figure...")
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
    axes0 = axes[0]
    axes1 = axes[1]

    im = axes0.imshow(np.flipud(E_mat), vmin=threshold_lower, vmax=threshold_upper, cmap="jet")
#    im = axes0.imshow(np.flipud(E_mat), vmin=-0.005, vmax=0.005, cmap="jet")
    axes0.set_xlabel('x')
    axes0.set_ylabel('y')

    im = axes1.imshow(np.flipud(B_mat), vmin=threshold_lower, vmax=threshold_upper, cmap="jet")
#    im = axes1.imshow(np.flipud(B_mat), vmin=-0.005, vmax=0.005, cmap="jet")
    axes1.set_xlabel('x')
    axes1.set_ylabel('y')

    fig.colorbar(im, ax=axes, orientation="horizontal")
    fig.suptitle("%s_%s | Rs: %.0f pix | Max abs: %.3e"%(tag0, tag, Rs_input, threshold) )

    print("Saving figure...")
    plt.savefig(f"{folder_name}/M_ap_map/{tag0}_Rs{int(Rs_input)}_bin{int(bin_size)}_{tag}.png")
    plt.close()


# ===========================================
# Define plot signal/noise map function
# ===========================================

def compute_SN_map_by_sigmae(mass_map_E, sigmae_bin, epsilon=1e-6):
    """
    用sigmae_bin（每格的测量误差）直接生成信噪比map。
    """
    return mass_map_E / (sigmae_bin + epsilon)


# ====================================
# Find and plot peaks definition
# ====================================


def find_SN_peaks(SN_map, threshold=5.0, footprint_size=3):
    """
    在S/N map中寻找超过阈值的局部峰。
    """
    footprint = np.ones((footprint_size, footprint_size))
    local_max = (SN_map == maximum_filter(SN_map, footprint=footprint))
    detected_peaks = local_max & (SN_map > threshold)
    labeled, num_features = label(detected_peaks)
    peak_coords = center_of_mass(SN_map, labeled, range(1, num_features + 1))
    peak_values = [SN_map[int(r), int(c)] for r, c in peak_coords]
    return peak_coords, peak_values


def plot_SN_map_with_peaks(SN_map, peak_coords, tag="SN Map", vmin=None, vmax=None):
    plt.figure(figsize=(6,5))
    im = plt.imshow(np.flipud(SN_map), cmap="hot", vmin=vmin, vmax=vmax)
    for y, x in peak_coords:
        plt.plot(x, SN_map.shape[0]-y, "bo", markersize=5)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"{tag} with Peaks")
    plt.colorbar(im, orientation="vertical", label="S/N")
    plt.tight_layout()
    plt.savefig(f"{folder_name}/SN_map_peaks/{tag0}_Rs{int(Rs_input)}_bin{int(bin_size)}_{tag}_peaks.png")
    plt.close()


# =====================================
# Compute and plot power spectrum
# =====================================

def compute_power_spectrum_2D(field):
    ft = np.fft.fft2(field)
    power = np.abs(ft)**2
    power = np.fft.fftshift(power)
    return power

def plot_power_spectrum(E_map, B_map, tag):
    E_power = compute_power_spectrum_2D(E_map)
    B_power = compute_power_spectrum_2D(B_map)

    # Log transform safely
    E_power_log = np.log10(E_power + 1e-10)
    B_power_log = np.log10(B_power + 1e-10)

    # Calculate finite vmax
    finite_vals = np.concatenate([E_power_log[np.isfinite(E_power_log)],
                                  B_power_log[np.isfinite(B_power_log)]])
    if len(finite_vals) == 0:
        print("Power spectrum contains no finite values. Skipping plot.")
        return

    vmin = np.percentile(finite_vals, 5)
    vmax = np.percentile(finite_vals, 99)

    # Ensure vmin ≤ vmax
    if vmin > vmax:
        vmin = vmax - 1
    if vmin == vmax:
        vmax = vmin + 1

    fig, axes = plt.subplots(1, 2, figsize = (12, 5), layout="constrained")
    im0 = axes[0].imshow(E_power_log, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0].set_title("E-mode Power Spectrum")
    im1 = axes[1].imshow(B_power_log, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[1].set_title("B-mode Power Spectrum")

    plt.suptitle(f"{tag0}_{tag} | Power Spectrum")
    plt.colorbar(im1, ax=axes, orientation="horizontal", pad=0.1)
    plt.savefig(f"{folder_name}/pwr_spec/{tag0}_Rs{int(Rs_input)}_bin{int(bin_size)}_{tag}_powerspec.png")
    plt.close()


# ==================================================
# Compute correlation coefficient of E and B
# ==================================================

def compute_EB_correlation(E_map, B_map):
    """
    Compute correlation coefficient of E and B after flattening the arrays.
    """
    mask = np.isfinite(E_map) & np.isfinite(B_map)
    corr = np.corrcoef(E_map[mask].flatten(), B_map[mask].flatten())[0, 1]
    print(f"Correlation coefficient of E and B: {corr}")
    return corr


# =====================================================================
# Define the execution function that runs the whole pipeline
# =====================================================================

# —— 1) 单滤波器处理函数 ——
def _process_one_filter(filter_name):
    # 1) 计算质量图
    M_ap_E, M_ap_B = calculate_mass_map(filter_name)

    # 2) 做 SN map
    SN_map = compute_SN_map_by_sigmae(M_ap_E, sigmae_backup, epsilon=1e-6)

    # 3) Kaiser–Squires 反演 κ map
    kappa_map = kaiser_squires_inversion(e1_backup, e2_backup)

    # 4) 计算 2D 功率谱
    E_power = compute_power_spectrum_2D(M_ap_E)
    B_power = compute_power_spectrum_2D(M_ap_B)

    # 5) 计算corr
    corr = compute_EB_correlation(M_ap_E, M_ap_B)

    return filter_name, {
        'M_ap_E'           : M_ap_E,
        'M_ap_B'           : M_ap_B,
        'kappa_map'        : kappa_map,
        'sn_map'           : SN_map,
        'power_spectrum_E' : E_power,
        'power_spectrum_B' : B_power,
        'corr'            : corr
    }

# —— 2) 并行版 run_all_filters ——
def run_all_filters_parallel():
    results = {}
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as exe:
        futures = {exe.submit(_process_one_filter, fname): fname for fname in filter_bank}
        for fut in as_completed(futures):
            fname = futures[fut]
            try:
                name, res = fut.result()
            except Exception as e:
                raise RuntimeError(f"线程池执行 {fname} 失败: {e}") from e
            results[name] = res
    return results



# ===============
# Save data
# ===============

def collect_results_to_df(
    filter_list,
    mass_map_E_dict,
    mass_map_B_dict,
    kappa_map_dict,
    sn_map_dict,
    power_spectrum_E_dict,
    power_spectrum_B_dict,
    corr_dict
    ):
    """
    将计算好的 mass map、κ map、S/N map、功率谱收集到一个 DataFrame。

    参数
    ----
    filter_list : list of str
        每组结果对应的标识（例如滤波器名称或 Rs 值）。
    mass_map_dict : dict
        key 为 filter 名称，value 为对应的 mass map（NumPy 数组）。
    kappa_map_dict : dict
        key 为 filter 名称，value 为对应的 κ map。
    sn_map_dict : dict
        key 为 filter 名称，value 为对应的 S/N map。
    power_spectrum_dict : dict
        key 为 filter 名称，value 为对应的功率谱。

    返回
    ----
    pd.DataFrame
        包含 columns=['filter_name', 'mass_map', 'kappa_map', 'sn_map', 'power_spectrum']。
    """
    records = []
    for fname in filter_list:
        records.append({
            'filter_name': fname,
            'mass_map_E': mass_map_E_dict[fname],
            'mass_map_B': mass_map_B_dict[fname],
            'kappa_map': kappa_map_dict[fname],
            'sn_map': sn_map_dict[fname],
            'power_spectrum_E': power_spectrum_E_dict[fname],
            'power_spectrum_B': power_spectrum_B_dict[fname],
            'corr': corr_dict[fname]
        })
    df_results = pd.DataFrame.from_records(
        records,
        columns=['filter_name', 'mass_map_E', 'mass_map_B', 'kappa_map', 'sn_map', 'power_spectrum_E', 'power_spectrum_B', 'corr']
    )
    return df_results

Plot filter functions

In [13]:
# Plotting filter weights
x = np.linspace(0, 2, 500)

plt.figure(figsize=(10, 6))

for filter_name, filter_func in filter_bank.items():
    if filter_name == "tophat":
        y = filter_func(x, smoothing_width=0.05)
        y = y * smooth_center_taper(x)
        y = normalize_filter(y)
    elif filter_name == "none":
        y = filter_func(x)
        y = normalize_filter(y)
    else:
        y = filter_func(x)
        y = y * smooth_center_taper(x)
        y = normalize_filter(y)

    plt.plot(x, y, label=filter_name)
    plt.tick_params(axis='x', labelsize=13)
    plt.tick_params(axis='y', labelsize=13)

plt.xlabel("r / R$_s$", fontsize = 16)
plt.ylabel("Weight (Q)", fontsize = 16)
plt.title("Filter Weights", fontsize = 18)
plt.legend(fontsize='large')
plt.grid(True)
plt.savefig(f"filter_weights.png")
plt.close()

Load csv files

In [14]:
filename_list = Table()
filename_list.add_column(Column(name='filename', data=[], dtype=str))
e_filter = 2.0
data_dict = {}

try:
  os.chdir(folder_path)
  print(f"Entered: {os.getcwd()}")
except FileNotFoundError:
  print(f"Folder '{folder_path}' does not exist.")

for filename_0 in file_names:

  suff = "data_"
  df_name = suff + filename_0.replace("-", "_")

  data = Table.read(filename_0 + ".csv", format="ascii.csv")
#  data['e1'] = np.where(np.isnan(data['e1']) | (data['e1'] == 'nan') | (data['e1'] == 'NaN'), 0, data['e1'])
#  data['e2'] = np.where(np.isnan(data['e2']) | (data['e2'] == 'nan') | (data['e2'] == 'NaN'), 0, data['e2'])
#  data = data[np.logical_and(data['e1'] != "nan", data['e2'] != "nan")]
#  data = data[np.logical_and(data['e1'] != "NaN", data['e2'] != "NaN")]
  data = data[np.logical_and(data['e1'] <= e_filter, data['e2'] <= e_filter)]
  data = data[np.logical_and(data['e1'] >= -e_filter, data['e2'] >= -e_filter)]
  data = data[data['r_cmodel_magerr'] <= 0.1]
#  data = data[data['z_cmodel_magerr'] <= 0.1]
  data = data[data['g_cmodel_magerr'] <= 0.1]
  data = data[data['i_cmodel_magerr'] <= 0.1]

  # 这里用SN筛选数据点
  data['sn'] = np.sqrt(data['e1']**2 + data['e2']**2) / data['sigmae']
  data = data[data['sn'] >= 3]

  exec(f"{df_name} = data")
  print(df_name)
  print(eval(df_name))
  filename_list.add_row([df_name])

  data_dict[df_name] = data

try:
    os.chdir(main_path)
    print(f"\nEntered: {os.getcwd()}")
except FileNotFoundError:
    print(f"Folder '{main_path}' does not exist.")

with open("all_data.pkl", "wb") as f:
    pickle.dump(data_dict, f)




Entered: c:\Users\skyma\Desktop\t1\csv
data_sep10000px
     ra          dec             x        ...  idn           sn        
------------ ------------ --------------- ... ------ ------------------
45.893802716 10.915306461 34411.023856912 ...     69 3.1153615053523125
46.007769537 12.614427103 35940.997818304 ...    103  3.647968483536329
43.357963898 10.350245177   368.068671705 ...    110   3.85694619508487
   44.706621  9.855777739 18473.428116247 ...    179 3.7741645493944462
43.891243374 11.627907664   7527.20243642 ...    180  3.063184834300974
43.742784461  9.881903202  5534.181388502 ...    199 3.0508942638684307
46.379346912  11.14429188 40929.324304388 ...    206  5.225603553775014
45.079179851 11.073605373 23474.930639389 ...    208  5.933219324132364
46.092437636 12.737204743 37077.644315396 ...    209  7.859427930843494
46.382500409  9.798582692 40971.659160442 ...    234 3.0405686066980246
         ...          ...             ... ...    ...                ...
 45.52573

If all data had been loaded and saved, run the code below directly

In [15]:
with open("all_data.pkl", "rb") as f:
    data_dict = pickle.load(f)
filename_list = list(data_dict.keys())
print(filename_list)

['data_sep10000px', 'data_sep12000px', 'data_sep2000px', 'data_sep4000px', 'data_sep6000px', 'data_sep8000px']


Plot maps

In [16]:
# ------------ Plot maps cell 开头  ------------
mass_map_E_dict = {}
mass_map_B_dict = {}
kappa_map_dict = {}
sn_map_dict = {}
power_spectrum_E_dict = {}
power_spectrum_B_dict = {}
corr_dict = {}

for filename in filename_list:

  for Rs_input in Rs_input_list:

    for bin_size in bin_size_list:

      Rs = Rs_input/bin_size

      filename_split = filename.split('/')
      if len(filename_split)==1:
        folder_name = '.'
      else:
        folder_name = filename_split[-2]
      tag0 = filename_split[-1].split('_')[1]

      print("")



#=======================
      x = data_dict[filename]["x"]
      y = data_dict[filename]["y"]
      e1 = data_dict[filename]["e1"]
      e2 = data_dict[filename]["e2"]
      #e1 = data["g1"] * 2.
      #e2 = data["g2"] * 2.
      sigmae = data_dict[filename]["sigmae"]


#-----------------------
      x_min = np.min(x)
      x_max = np.max(x)

      y_min = np.min(y)
      y_max = np.max(y)

      #print(x_min, x_max, y_min, y_max)

      ncol = int(np.ceil((x_max - x_min)/bin_size))
      nrow = int(np.ceil((y_max - y_min)/bin_size))
      print(nrow, ncol)

# x, y start from 0
# so that x -> col, y -> row
# Note the direction! Different from above
# xv:
# 0 1 2
# 0 1 2
# 0 1 2
# yv:
# 0 0 0
# 1 1 1
# 2 2 2
      xv, yv = np.meshgrid(np.arange(ncol), np.arange(nrow))

# [ (Row, Col) ]
      coord_list = list(zip(yv.flatten(), xv.flatten() ) )

#-----------------------
# START calculation

      print("Running bin stat...")
      e1_binned = get_bin_stat(e1)
      e2_binned = get_bin_stat(e2)
      sigmae_binned = bin_sigmae(sigmae)

      e1_backup = e1_binned.copy()
      e2_backup = e2_binned.copy()
      sigmae_backup = sigmae_binned.copy()

#-----------------------
# —— 调用修改后的 run_all_filters ——
      print(f"Processing {filename}, Rs input={Rs_input}, bin={bin_size} …")
      print(f"RS(滤波器半径)为：{Rs}")
      # 先做并行计算
      results = run_all_filters_parallel()
      # results = run_all_filters()

      # 然后统一绘图
      for filter_name, res in results.items():
          # E/B 质量图
          plot_E_B(
              res['M_ap_E'],
              res['M_ap_B'],
              tag=f"M_ap_{filter_name}"
          )
          # 在 S/N map 上标 Peaks
          peak_coords, _ = find_SN_peaks(res['sn_map'], threshold=5.0)
          plot_SN_map_with_peaks(
              res['sn_map'],
              peak_coords,
              tag=f"SN_peaks_{filter_name}"
          )
          # Kappa map
          plot_kappa_map(
              res['kappa_map'],
              tag=f"Kappa_{filename}"
          )
          # 功率谱
          plot_power_spectrum(
              res['M_ap_E'],
              res['M_ap_B'],
              tag=f"PowerSpec_{filter_name}"
          )
          print(f"Plotted all maps for {filter_name}\n" + "-"*40)

# —— 将本次循环中每个滤波器的结果，存进对应的全局字典 ——
      tagg = f"{filename.split('/')[-1].split('_')[1]}_Rs{Rs_input}_bin{bin_size}"
      for fname, res in results.items():
          key = f"{tagg}_{fname}"
          mass_map_E_dict[key] = res['M_ap_E']
          mass_map_B_dict[key] = res['M_ap_B']
          kappa_map_dict[key] = res['kappa_map']
          sn_map_dict[key] = res['sn_map']
          power_spectrum_E_dict[key] = res['power_spectrum_E']
          power_spectrum_B_dict[key] = res['power_spectrum_B']
          corr_dict[key] = res['corr']

# END of M_ap calculation!


83 83
Running bin stat...
Processing data_sep10000px, Rs input=5000.0, bin=500 …
RS(滤波器半径)为：10.0
Using filter: schirmer
Using filter: gaussian
Using filter: shifted_gaussian
Using filter: tophat
Using filter: nfw
Using filter: none
Correlation coefficient of E and B: -0.11140558796893892
Correlation coefficient of E and B: 0.0037426237083744932
Correlation coefficient of E and B: 0.0019731235777818643
Correlation coefficient of E and B: 0.005477607772897011
Correlation coefficient of E and B: 0.006939614075349445
Correlation coefficient of E and B: -0.01251160325871921
Plotting E B figure...
Saving figure...
Plotted all maps for none
----------------------------------------
Plotting E B figure...
Saving figure...
Plotted all maps for gaussian
----------------------------------------
Plotting E B figure...
Saving figure...
Plotted all maps for shifted_gaussian
----------------------------------------
Plotting E B figure...
Saving figure...
Plotted all maps for tophat
------------------